# AGE Samples 

## Prepare
```
import age
```
## Connect to PostgreSQL(with AGE extension)
* Connect to PostgreSQL server 
* Load AGE and register agtype to db session (Psycopg2 driver)
* Check graph exists and set graph. If not, age make that.

```
ag = age.connect(graph="(graph name}", host="{host}", port="{port}", dbname="{dbname}", user="{db username}", password="{password}")

# or 
DSN = "host={host} port={port} dbname={dbname} user={db username} password={password}"
ag = age.connect(graph="(graph name}", dsn=DSN)

# or Without Graph Name : you can make a new graph later.

ag = age.connect(host="{host}", port="{port}", dbname="{dbname}", user="{db username}", password="{password}")

# And set graph - if you don't have one yet, setGraph make that.)
ag = age.setGraph("{graph name}")
```

In [1]:
import age
from age.gen.AgtypeParser import *

GRAPH_NAME = "test_graph"
DSN = "host=172.17.0.2 port=5432 dbname=postgres user=postgres password=agens"

ag = age.connect(graph=GRAPH_NAME, dsn=DSN)


---
# API

### age.connect(graph:str=None, dsn:str=None, connection_factory=None, cursor_factory=None, **kwargs) -> Age
> Connect PostgreSQL server 
  Parameters : dsn={dsn} or 
  host="{host}", port="{port}", dbname="{dbname}", user="{db username}", password="{password}"

### Age.commit() , Age.rollback()
> If your statement change data, you must call 'Age.commit()' explicitly. Otherwise change will not make effect.
> Or when execution error occurs, you must call 'Age.rollback()'

### Age.close()
> Closes connection to PostgreSQL.

### Age.execCypher(cypherStmt:str, cols:list=None, params:tuple=None) -> psycopg2.extensions.cursor :
>  Execute cypher statements to query or change data (CREATE, SET, REMOVE) with or without result.
>  If your statement change data, you must call 'Age.commit()' explicitly. Otherwise change will not make effect.
    
>  If your execution returns no result or only one result, you don't have to set 'cols' argument.
>  But it returns many columns, you have to pass columns names(and types) to 'cols' argument.

>  cols : str list \[ 'colName {type}', ... \] : If column data type is not set, agtype is default.
        
### Age.cypher(cursor:psycopg2.extensions.cursor, cypherStmt:str, cols:list=None, params:tuple=None) -> psycopg2.extensions.cursor :
>  If you want execute many statements (changing data statement maybe) with one transaction explicitly, you may use Age.cypher(...) function.

>  For creating cursor and mamage transaction, you usually use 'with' clause.
    
>  If your execution returns no result or only one result, you don't have to set 'cols' argument.
>  But it returns many columns, you have to pass columns names(and types) to 'cols' argument.

>  cols : str list \[ 'colName {type}', ... \] : If column data type is not set, agtype is default.
        

---
## Create  & Change Vertices

> If cypher statement changes data (create, set, remove), 
  you must use execCypher(cypherStmt, commit, *args). 
  
>  If **'commit'** argument is **True**: the cypherStmt make effect automatically, but cursor is closed after execution. So you cannot access the result.  
  If **False** : you can access the result, but you must commit session(ag.commit()) explicitly.
  (Otherwise the execution cannot make any effect.)


> execCypher(cypherStmt:str, commit:bool, *args) 

```
cursor = ag.execCypher("CREATE(...)", commit=False) # Cypher Create Statement
...
# check result in cursor
...
ag.commit() # commit explicitly
```


In [2]:
# Create Vertices
ag.execCypher("CREATE (n:Person {name: 'Joe'})")
ag.execCypher("CREATE (n:Person {name: 'Smith'})")
    
# Execution with one agtype result
cursor = ag.execCypher("CREATE (n:Person {name: %s}) RETURN n", params=('Jack',))
for row in cursor:
    print("CREATED: ", row[0]) 
    
cursor = ag.execCypher("CREATE (n:Person {name: %s, title: 'Developer'}) RETURN id(n)", params=('Andy',))
for row in cursor:
    print("CREATED: ", row[0])
    

# Execution with one result as SQL TYPE 
cursor = ag.execCypher("MATCH (n:Person {name: %s}) SET n.title=%s RETURN n.title", cols=["a VARCHAR"], params=('Smith','Manager',))
for row in cursor:
    print("SET: ", row[0])


# Execution with one result as SQL TYPE 
cursor = ag.execCypher("MATCH (n:Person {name: %s}) REMOVE n.title RETURN id(n)", cols=["a BIGINT"], params=('Smith',))
for row in cursor:
    print("REMOVE Prop: ", row[0])

# You must commit explicitly
ag.commit()


CREATED:  {label:Person, id:844424930131971, properties:{name: Jack}}::VERTEX
CREATED:  844424930131972
SET:  "Manager"
REMOVE Prop:  844424930131970


---
## Query Vertices

> execCypher(cypherStmt:str, cols:list=None, params:tuple=None) 

### Single result column

```
cursor = ag.execCypher("MATCH (n:Person {name: %s) RETURN n", params('Andy',))
for row in cursor:
    vertex = row[0]
    print(vertex.id, vertex["name"], vertex) # row has id, label, properties 
```

### Multi result columns

```
cursor = ag.execCypher("MATCH (n:Person) RETURN label(n), n.name", cols=['label VARCHAR', 'name'])
for row in cursor:
    label = row[0]
    name = row[1]
    print(label, name) 
```


### Vertex object has id, label attribute and __getitem__, __setitem__ for properties
```
vertex.id
vertex.label
vertex["property_name"]
```

In [3]:

# Query Vertices with parsed row cursor.
print("-- Query Vertices  --------------------")
cursor = ag.execCypher("MATCH (n:Person) RETURN n")
for row in cursor:
    vertex = row[0]
    print(vertex.id, vertex.label, vertex["name"])
    print("-->", vertex)

# Query Vertices with with multi column
print("-- Query Vertices with with multi columns. --------------------")
cursor = ag.execCypher("MATCH (n:Person) RETURN label(n), n.name", cols=['label VARCHAR', 'name'])
for row in cursor:
    label = row[0]
    name = row[1]
    print(label, name) 


-- Query Vertices  --------------------
844424930131969 Person Joe
--> {label:Person, id:844424930131969, properties:{name: Joe}}::VERTEX
844424930131971 Person Jack
--> {label:Person, id:844424930131971, properties:{name: Jack}}::VERTEX
844424930131972 Person Andy
--> {label:Person, id:844424930131972, properties:{name: Andy, title: Developer}}::VERTEX
844424930131970 Person Smith
--> {label:Person, id:844424930131970, properties:{name: Smith}}::VERTEX
-- Query Vertices with with multi columns. --------------------
"Person" Joe
"Person" Jack
"Person" Andy
"Person" Smith


---
## Create Relation 

> execCypher(cypherStmt:str, commit:bool, *args)


```
# Execute statement and handle results
cursor = ag.execCypher("MATCH (a:Person), (b:Person) WHERE  a.name = %s AND b.name = %s CREATE p=((a)-[r:workWith]->(b)) RETURN p", False, ('Andy', 'Smith',))
...
# You can access the results in cursor
...
ag.commit() # commit
```

```
# Auto commit
ag.execCypher("MATCH (a:Person), (b:Person) WHERE  a.name = 'Andy' AND b.name = 'Tom' CREATE (a)-[r:workWith]->(b)", True)

```


In [4]:

# Create Edges
ag.execCypher("MATCH (a:Person), (b:Person) WHERE a.name = 'Joe' AND b.name = 'Smith' CREATE (a)-[r:workWith {weight: 3}]->(b)")
ag.execCypher("MATCH (a:Person), (b:Person) WHERE  a.name = 'Andy' AND b.name = 'Tom' CREATE (a)-[r:workWith {weight: 1}]->(b)")
ag.execCypher("MATCH (a:Person {name: 'Jack'}), (b:Person {name: 'Andy'}) CREATE (a)-[r:workWith {weight: 5}]->(b)")

ag.commit()

# With Params and Return
cursor = ag.execCypher("""MATCH (a:Person), (b:Person) 
        WHERE  a.name = %s AND b.name = %s 
        CREATE p=((a)-[r:workWith]->(b)) 
        RETURN p""",  
        params=('Andy', 'Smith',))

for row in cursor:
    print(row[0])

ag.commit()

# With many columns Return
cursor = ag.execCypher("""MATCH (a:Person {name: 'Joe'}), (b:Person {name: 'Jack'}) 
        CREATE (a)-[r:workWith {weight: 5}]->(b) 
        RETURN a, r, b """, cols=['a','r', 'b'])

for row in cursor:
    print("(a)", row[0], ": (r)", row[1], ": (b)", row[2])
    

ag.commit()

    

[{label:Person, id:844424930131972, properties:{name: Andy, title: Developer}}::VERTEX,{label:workWith, id:1125899906842627, properties:{}, start_id:844424930131972, end_id:844424930131970}::EDGE,{label:Person, id:844424930131970, properties:{name: Smith}}::VERTEX]::PATH
(a) {label:Person, id:844424930131969, properties:{name: Joe}}::VERTEX : (r) {label:workWith, id:1125899906842628, properties:{weight: 5}, start_id:844424930131969, end_id:844424930131971}::EDGE : (b) {label:Person, id:844424930131971, properties:{name: Jack}}::VERTEX


---
## Query Relations

> With single column
```
cursor = ag.execCypher("MATCH p=()-[:workWith]-() RETURN p")
for row in cursor:
    path = row[0]
    print(path)  
```

> With multi columns
```
cursor = ag.execCypher("MATCH p=(a)-[b]-(c) RETURN a,label(b),c", cols=["a","b VARCHAR","c"])
for row in cursor:
    start = row[0]
    edge = row[1]
    end = row[2]
    print(start["name"] , edge.label, end["name"])  
```


### Edge object has id, label,start_id, end_id attribute and __getitem__, __setitem__ for properties
```
edge = path.rel
edge.id
edge.label
edge.start_id
edge.end_id
edge["property_name"]
edge.properties
```

In [7]:
cursor = ag.execCypher("MATCH p=()-[:workWith]-() RETURN p")
for row in cursor:
    path = row[0]
    print("START:", path[0])
    print("EDGE:", path[1])
    print("END:", path[2])  

print("-- [Query path with multi columns --------")
cursor = ag.execCypher("MATCH p=(a)-[b]-(c) WHERE b.weight>%s RETURN a,label(b), b.weight, c", cols=["a","bl","bw", "c"], params=(2,))
for row in cursor:
    start = row[0]
    edgel = row[1]
    edgew = row[2]
    end = row[3]
    print(start["name"] , edgel, edgew, end["name"]) 


    

START: {label:Person, id:844424930131971, properties:{name: Jack}}::VERTEX
EDGE: {label:workWith, id:1125899906842628, properties:{weight: 5}, start_id:844424930131969, end_id:844424930131971}::EDGE
END: {label:Person, id:844424930131969, properties:{name: Joe}}::VERTEX
START: {label:Person, id:844424930131970, properties:{name: Smith}}::VERTEX
EDGE: {label:workWith, id:1125899906842625, properties:{weight: 3}, start_id:844424930131969, end_id:844424930131970}::EDGE
END: {label:Person, id:844424930131969, properties:{name: Joe}}::VERTEX
START: {label:Person, id:844424930131969, properties:{name: Joe}}::VERTEX
EDGE: {label:workWith, id:1125899906842628, properties:{weight: 5}, start_id:844424930131969, end_id:844424930131971}::EDGE
END: {label:Person, id:844424930131971, properties:{name: Jack}}::VERTEX
START: {label:Person, id:844424930131972, properties:{name: Andy, title: Developer}}::VERTEX
EDGE: {label:workWith, id:1125899906842626, properties:{weight: 5}, start_id:844424930131971,

---
## Query COLLECT


In [8]:

with ag.connection.cursor() as cursor:
    ag.cypher(cursor, "MATCH (a)-[:workWith]-(c) WITH a as V, COLLECT(c) as CV RETURN V.name, CV", cols=["V","CV"])
    for row in cursor:
        nm = row[0]
        collected = row[1]
        print(nm, "workWith", [i["name"] for i in collected])

for row in ag.execCypher("MATCH (a)-[:workWith]-(c) WITH a as V, COLLECT(c) as CV RETURN V.name, CV", cols=["V1","CV"]):
    nm = row[0]
    collected = row[1]
    print(nm, "workWith", [i["name"] for i in collected])


Joe workWith ['Jack', 'Smith']
Smith workWith ['Joe', 'Andy']
Jack workWith ['Joe', 'Andy']
Andy workWith ['Jack', 'Smith']
Joe workWith ['Jack', 'Smith']
Smith workWith ['Joe', 'Andy']
Jack workWith ['Joe', 'Andy']
Andy workWith ['Jack', 'Smith']


--- 
## Many executions in one transaction & Multiple Edges

In [9]:
with ag.connection.cursor() as cursor:
    try :
        ag.cypher(cursor, "CREATE (n:Country {name: %s}) ", params=('USA',))
        ag.cypher(cursor, "CREATE (n:Country {name: %s}) ", params=('France',))
        ag.cypher(cursor, "CREATE (n:Country {name: %s}) ", params=('Korea',))
        ag.cypher(cursor, "CREATE (n:Country {name: %s}) ", params=('Russia',))

        # You must commit explicitly after all executions.
        ag.connection.commit()
    except Exception as ex:
        ag.rollback()
        raise ex

with ag.connection.cursor() as cursor:
    try :# Create Edges
        ag.cypher(cursor,"MATCH (a:Country), (b:Country) WHERE a.name = 'USA' AND b.name = 'France' CREATE (a)-[r:distance {unit:'miles', value: 4760}]->(b)")
        ag.cypher(cursor,"MATCH (a:Country), (b:Country) WHERE  a.name = 'France' AND b.name = 'Korea' CREATE (a)-[r:distance {unit: 'km', value: 9228}]->(b)")
        ag.cypher(cursor,"MATCH (a:Country {name: 'Korea'}), (b:Country {name: 'Russia'}) CREATE (a)-[r:distance {unit:'km', value: 3078}]->(b)")

        # You must commit explicitly
        ag.connection.commit()
    except Exception as ex:
        ag.rollback()
        raise ex


cursor = ag.execCypher("""MATCH p=(:Country {name:"USA"})-[:distance]-(:Country)-[:distance]-(:Country) 
        RETURN p""")

for row in cursor:
    path = row[0]
    indent = ""
    for e in path:
        if e.gtype == age.TP_VERTEX:
            print(indent, e.label, e["name"])
        elif e.gtype == age.TP_EDGE:
            print(indent, e.label, e["value"], e["unit"])
        else:
            print(indent, "Unknown element.", e)
            
        indent += " >"


 Country USA
 > distance 4760 miles
 > > Country France
 > > > distance 9228 km
 > > > > Country Korea


---
## Query Scalar or properties value

In [10]:
# Query scalar value
print("-- Query scalar value --------------------")
for row in ag.execCypher("MATCH (n:Person) RETURN id(n)"):
    print(row[0])
    
# Query properties 
print("-- Query properties --------------------")

for row in ag.execCypher("MATCH (n:Person) RETURN properties(n)"):
    print(row[0])
    
# Query properties value
print("-- Query property value --------------------")
for row in ag.execCypher("MATCH (n:Person {name: 'Andy'}) RETURN n.title"):
    print(row[0])
    
   

-- Query scalar value --------------------
844424930131969
844424930131971
844424930131972
844424930131970
-- Query properties --------------------
{'name': 'Joe'}
{'name': 'Jack'}
{'name': 'Andy', 'title': 'Developer'}
{'name': 'Smith'}
-- Query property value --------------------
Developer


## Close connection

In [11]:
# Clear test data
age.deleteGraph(ag.connection, GRAPH_NAME)
# connection close
ag.close()